In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Arrhythmia/GSE136992'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"mRNA expression in SIDS"
!Series_summary	"Genetic predispositions in cases suffering sudden unexpected infant death have been a research focus worldwide the last decade. Despite large efforts there is still uncertainty concerning the molecular pathogenesis of these deaths. With genetic technology in constant development the possibility of an alternative approach into this research field have become available, like mRNA expression studies.  Methods: In this study we investigated mRNA gene expression in 14 cases that died suddenly and unexpectedly from infection without a history of severe illness prior to death. The control group included eight accidents, two cases of natural death, one undetermined, one case of medical malpractice and two homicides. The study included tissue from liver, heart and brain. The mRNA expression was determined using Illumina whole genome gene expression DASL HT assay.  Results: From the array, 19 genes showed altered ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check availability of 'Arrhythmia' trait, age, and gender data
trait_row = 0  # Assuming 'condition' can correlate to 'Arrhythmia'
age_row = 2  # Based on given sample characteristics, data is present in row 2
gender_row = 3  # Based on given sample characteristics, data is present in row 3

# Define function to convert condition to binary, assuming 
# 'Infection': 1 (associated with Arrhythmia risk), 'Control': 0
def convert_trait(value):
    condition = value.split(':')[1].strip().lower()
    if condition == 'infection':
        return 1
    elif condition == 'control':
        return 0
    else:
        return None

# Define function to convert age to continuous value in weeks
def convert_age(value):
    try:
        age_in_weeks = float(value.split(':')[1].strip().split(' ')[0])
        return age_in_weeks
    except Exception as e:
        return None

# Define function to convert gender to binary, female: 0, male: 1
def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE136992', './preprocessed/Arrhythmia/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical features if trait_row is defined
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Arrhythmia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Arrhythmia/trait_data/GSE136992.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4064970': [1.0, 24.0, 1.0], 'GSM4064971': [1.0, 112.0, 1.0], 'GSM4064972': [1.0, 8.0, 0.0], 'GSM4064973': [1.0, 24.0, 1.0], 'GSM4064974': [1.0, 0.6, 0.0], 'GSM4064975': [1.0, 72.0, 1.0], 'GSM4064976': [1.0, 24.0, 0.0], 'GSM4064977': [1.0, 36.0, 1.0], 'GSM4064978': [1.0, 52.0, 1.0], 'GSM4064979': [1.0, 20.0, 1.0], 'GSM4064980': [0.0, 24.0, 0.0], 'GSM4064981': [0.0, 0.0, 0.0], 'GSM4064982': [0.0, 0.0, 0.0], 'GSM4064983': [0.0, 80.0, 0.0], 'GSM4064984': [0.0, 52.0, 0.0], 'GSM4064985': [0.0, 0.5, 0.0], 'GSM4064986': [0.0, 144.0, 1.0], 'GSM4064987': [0.0, 0.0, 1.0], 'GSM4064988': [0.0, 24.0, 1.0], 'GSM4064989': [0.0, 0.0, 1.0], 'GSM4064990': [1.0, 112.0, 1.0], 'GSM4064991': [1.0, 24.0, 1.0], 'GSM4064992': [1.0, 8.0, 0.0], 'GSM4064993': [1.0, 0.6, 0.0], 'GSM4064994': [1.0, 20.0, 1.0], 'GSM4064995': [1.0, 36.0, 1.0], 'GSM4064996': [1.0, 12.0, 0.0], 'GSM4064997': [1.0, 72.0, 1.0], 'GSM4064998': [1.0, 0.0, 0.0], 'GSM4064999': [1.0, 52.0, 1.0], 'GSM4065000': [1.0, 24.0, 1.0], 'GSM4065001': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Addres

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and update the gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Arrhythmia/gene_data/GSE136992.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Arrhythmia')

# 4. Save the cohort information.
json_path = './preprocessed/Arrhythmia/cohort_info.json'
save_cohort_info('GSE136992', json_path, True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Arrhythmia/GSE136992.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Arrhythmia', the least common label is '0.0' with 29 occurrences. This represents 48.33% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.375
  50% (Median): 24.0
  75%: 52.0
Min: 0.0
Max: 144.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

